In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from mpi4py import MPI
from eigentools import Eigenproblem, CriticalFinder
import dedalus.public as de
import numpy as np
import time

In [10]:
# relevant parameters
_llambda = 1
_l = 0.1
_W = 1
_gd = 1
_tau = 1
_k = 1
_eta = 1
_a = 0.1

_aux_const = _k*_k*_llambda / (1 + (_gd * _tau)**2)

In [12]:
# resolution
ny = 58

# define spectral representation of the solution
y = de.Chebyshev('y',ny,interval=[-1/2,1/2])
d = de.Domain([y], grid_dtype=np.complex128)

# initialize problem
evp = de.EVP(d, ['psi','psi1','psi2','psi3','Qxx','Qxx1','Qxy','Qxy1'],'sigma')

# define paramters 
evp.parameters['_llambda'] = _llambda
evp.parameters['_eta'] = _eta
evp.parameters['_l'] = _l
evp.parameters['_W'] = _W
evp.parameters['_gd'] = _gd
evp.parameters['_tau'] = _tau
evp.parameters['_k'] = _k
evp.parameters['_eta'] = _eta
evp.parameters['_a'] = _a
evp.parameters['_aux_const'] = _aux_const

# define equations
evp.add_equation("1/_tau * (_k**4 * psi - 2*(_k**2)*psi2 + dy(psi3)) + 2j*_a*_k*Qxx/_eta - (_a*_k*_k*Qxy + _a*dy(Qxy1)) = 0 ")
evp.add_equation("_aux_const*_gd*_tau*psi - 2j*_k*_llambda*psi1 - _gd*_llambda*_tau/(1+(_gd*_tau)**2)*psi2"\
                "+ (1 + (_l*_k)**2)/(_W**2) + 1j*_k*y*_gd*_tau*Qxx - (_l**2/_W**2)*dy(Qxx1)"\
                "-_gd*_tau*Qxy + sigma*_tau*Qxy = 0")
evp.add_equation("_aux_const*(1+2*(_gd*_tau)**2)*psi - _llambda/(1+(_gd+_tau)**2)*psi2"\
                 "+ _gd*_tau*Qxx"\
                 "+ (1 + (_l**2 * _k**2)/_W**2 + 1j*_k*y*_gd*_tau - (_l**2/_W**2)*dy(Qxy1))"\
                "- (_l**2/_W**2)*dy(Qxy1) + sigma*_tau*Qxy= 0")
evp.add_equation("dy(psi2) - Psi3 = 0")
evp.add_equation("dy(psi1) - psi2 = 0")
evp.add_equation("dy(psi) - psi1 = 0")
evp.add_equation("dy(Qxx) - Qxx1 = 0")
evp.add_equation("dy(Qxy) - Qxy1 = 0")

# impose boundary conditions
evp.add_bc('left(psi) = 0')
evp.add_bc('right(psi) = 0')
evp.add_bc('left(psi1) = 0')
evp.add_bc('right(psi1) = 0')
evp.add_bc('left(Qxx) = 0')
evp.add_bc('right(Qxx) = 0')
evp.add_bc('left(Qxy) = 0')
evp.add_bc('right(Qxy) = 0')

# create eigenproblem object
EP = Eigenproblem(evp)

2022-12-11 23:55:54,355 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


NonlinearOperatorError: Cannot add dependent and independent terms.